In [1]:
## Importação das bibliotecas

import pandas as pd
import numpy as np
import seaborn as sns
import plotly 
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
import copy
from IPython.display import display
import warnings
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
warnings.filterwarnings("ignore")
plotly.offline.init_notebook_mode (connected = True)

In [2]:
## Importação dos datasets

df = pd.read_csv(r"C:\TCC\Datasets\data.csv", delimiter =',', encoding='latin-1')

In [5]:
df.dtypes

acousticness        float64
artists              object
danceability        float64
duration_ms           int64
energy              float64
explicit              int64
id                   object
instrumentalness    float64
key                   int64
liveness            float64
loudness            float64
mode                  int64
name                 object
popularity            int64
release_date         object
speechiness         float64
tempo               float64
valence             float64
year                  int64
dtype: object

In [3]:
## Dropa colunas não utilizadas

df.drop(columns=['id','release_date'],inplace=True)


## Padroniza as colunas

x=df[df.drop(columns=['artists','name']).columns].values
scaler =StandardScaler().fit(x)
X_scaled = scaler.transform(x)
df[df.drop(columns=['artists','name']).columns]=X_scaled


## Ordena os valores da coluna

df.sort_values('tempo',inplace=True)

In [7]:
## Função de recomendação

def busca_musica(musica, df, number=10):
    df.drop_duplicates(inplace=True)
    musicas=df['name'].values
    artistas=df['artists'].values
    t=[]
    count=0
    if musica[-1]==' ':
        musica=musica[:-1]
    for i in musicas:
        if musica.lower() in i.lower():
            t.append([len(musica)/len(i),count])
        else:
            t.append([0,count])
        count+=1
    t.sort(reverse=True)
    s=[[musicas[t[i][1]],artistas[t[i][1]].strip('][').split(', ')] for i in range(number)]   
    lista_musicas=[musicas[t[i][1]] for i in range(number)]
    artista=[artistas[t[i][1]] for i in range(number)]
    x=[]
    for i in s:
        l=''
        by=''
        for j in i[1]:
            by+=j
        l+=i[0]+' by '+by
        x.append(l)
    tup=[]
    for i in range(number):
        tup.append((x[i],i))

    
    return tup,lista_musicas,artista

In [8]:
def mat_corr(df,best,number,artista):
    df.drop_duplicates(subset=['artists','name'],inplace=True)
    x=df[(df['name']==best) & (df['artists']==artista)].drop(columns=['name','artists']).values
    artista=artista.replace("'","").replace("'","").replace('[','').replace(']','')
    if ',' in artista:
        inm = artista.rfind(",")
        artista=artista[:inm]+' and'+artista[inm+1:]
    print('A música selecionada é:','"',best,'"',' por ','"',artista,'"')
    

    song_names=df['name'].values
    p=[]
    count=0
    for i in df.drop(columns=['artists','name']).values:
        p.append([distance.correlation(x,i),count])
        count+=1
    p.sort()
    for i in range(1,number+1):
        artistas=df['artists'].values
        artista=artistas[p[i][1]]
        artista=artista.replace("'","").replace("'","").replace('[','').replace(']','')
        if ',' in artista:
            inm = artista.rfind(",")
            artista=artista[:inm]+' and'+artista[inm+1:]
        print(song_names[p[i][1]],'by',artista)

In [9]:
a=input('Insira o nome da música:')
b=int(input('Quantas recomendações você precisa? '))
tup,s,ar=busca_musica(a,df)
ans=widgets.Dropdown(
    options=tup,
    description='Selecione:'+a,
    disabled=False,
)
display(ans)

Insira o nome da música:Baby
Quantas recomendações você precisa? 5


Dropdown(description='Selecione:Baby', options=(("Baby by 'Ashanti'", 0), ("Baby by 'Os Mutantes'", 1), ("Baby…

In [10]:
mat_corr(df,s[ans.value],b,ar[ans.value])

A música selecionada é: " Baby "  por  " Ashanti "
A Thousand Times by Ella Mai
Ghetto Angels by NoCap
Fuck You Tonight (feat. R. Kelly) - 2014 Remaster by The Notorious B.I.G. and R. Kelly
Feelins by PnB Rock
Ballin' by Jeezy and Lil Wayne
